# 1. Load Data To Snowflake
### This notebook uploads the training data to Snowflake
In a real scenario the data would already be in Snowflake

In [25]:
import pandas as pd
from snowflake.snowpark.session import Session
import configparser


In [26]:
telco_df = pd.read_parquet("../test_data/raw_telco_data.parquet")

In [27]:
telco_df.head()

,CUSTOMERID,GENDER,PARTNER,DEPENDENTS,COUNTRY,CITY,STATE,ZIP CODE,LONGITUDE,LATITUDE,...,STREAMING TV,CONTRACT,CHURN SCORE,ONLINE BACKUP,DEVICE PROTECTION,TOTAL CHARGES,CLTV,CHURN VALUE,CHURN LABEL,CHURN REASON
0,7090-ZyCMx,Female,False,True,United States,Los Angeles,California,90005,-118.307420,34.059281,...,No,Month-to-month,1,No,No,151.65,2701,1.0,True,Moved
1,1364-wJXMS,Female,False,True,United States,Los Angeles,California,90006,-118.293953,34.048013,...,Yes,Month-to-month,1,No,Yes,820.50,5372,1.0,True,Moved
2,6564-sLgIC,Male,False,True,United States,Los Angeles,California,90065,-118.229715,34.108833,...,Yes,Month-to-month,1,No,No,1752.55,3179,1.0,True,Competitor made better offer
3,7853-2xheR,Male,False,True,United States,La Habra,California,90631,-117.951300,33.940619,...,No,Month-to-month,1,No,No,229.55,4415,1.0,True,Product dissatisfaction
4,8457-E9FuW,Female,False,True,United States,Glendale,California,91206,-118.203869,34.162515,...,No,Month-to-month,1,Yes,Yes,1111.65,5142,1.0,True,Price too high


In [28]:
telco_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 33 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CUSTOMERID         100000 non-null  object 
 1   GENDER             100000 non-null  object 
 2   PARTNER            100000 non-null  bool   
 3   DEPENDENTS         100000 non-null  bool   
 4   COUNTRY            100000 non-null  object 
 5   CITY               100000 non-null  object 
 6   STATE              100000 non-null  object 
 7   ZIP CODE           100000 non-null  object 
 8   LONGITUDE          100000 non-null  object 
 9   LATITUDE           100000 non-null  object 
 10  PHONE SERVICE      100000 non-null  object 
 11  MULTIPLE LINES     100000 non-null  object 
 12  ONLINE SECURITY    100000 non-null  object 
 13  SENIOR CITIZEN     100000 non-null  bool   
 14  MONTHLY CHARGES    100000 non-null  float64
 15  STREAMING MOVIES   100000 non-null  object 
 16  PAY

In [29]:
telco_df.describe()

,MONTHLY CHARGES,TOTAL CHARGES,CHURN VALUE
count,100000.000000,100000.000000,100000.000000
mean,65.601117,2222.032082,0.322260
std,29.808447,2248.711822,0.467344
min,18.250000,0.000000,0.000000
25%,40.200000,357.750000,0.000000
50%,71.100000,1341.500000,0.000000
75%,90.100000,3653.000000,1.000000
max,118.750000,8684.800000,1.000000


In [30]:
# there are no duplicated customer ids, so we can split the data however we like
len(telco_df['CUSTOMERID'].unique())

100000

In [31]:
# Split the data into two sets: 
# 1. The initial training dataset (labeled)
# 2. The set to make predictions on (simulates live data, not labeled)

train_df = telco_df.iloc[0:90000]
live_df = telco_df.iloc[90000:-1]


train_df.to_parquet("../test_data/telco_labeled_subset.parquet")

live_df.pop('CHURN VALUE')
live_df.pop('CHURN LABEL')
live_df.pop('CHURN REASON')

live_df.to_parquet("../test_data/telco_unlabeled_subset.parquet")

In [32]:

config = configparser.ConfigParser()
config.read('../sf_account.config')
session = Session.builder.configs(dict(config['DEFAULT'])).create()  


In [33]:
# object setup
# (make sure your role in sf_account.config has the necessary privileges)
database = "CUSTOMER_DB"
schema = "CHURN"
stage = "DATA_LOADING"

In [23]:
# Create Objects and Push to Snowflake



session.sql(f"create database if not exists {database}").collect()
session.sql(f"create schema if not exists {database}.{schema}").collect()
session.sql(f"create stage if not exists {database}.{schema}.{stage} DIRECTORY = (ENABLE = TRUE)").collect()

# load the files
session.sql(f"CREATE OR REPLACE FILE FORMAT {database}.{schema}.CHURN_PARQUET_FORMAT TYPE = PARQUET;").collect()


session.file.put("../test_data/telco_labeled_subset.parquet", stage)
session.file.put("../test_data/telco_unlabeled_subset.parquet", stage)

session.sql(f"CREATE OR REPLACE \
            TABLE {database}.{schema}.TELCO_LABELED USING TEMPLATE ( \
                SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*)) \
                FROM \
                    TABLE( INFER_SCHEMA( \
                    LOCATION => '@{stage}/telco_labeled_subset.parquet', \
                    FILE_FORMAT => 'CHURN_PARQUET_FORMAT' \
                    ) \
                ) \
            );  ").collect()

session.sql(f"CREATE OR REPLACE \
            TABLE {database}.{schema}.TELCO_UNLABELED USING TEMPLATE ( \
                SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*)) \
                FROM \
                    TABLE( INFER_SCHEMA( \
                    LOCATION => '@{stage}/telco_unlabeled_subset.parquet', \
                    FILE_FORMAT => 'CHURN_PARQUET_FORMAT' \
                    ) \
                ) \
            );  ").collect()




[Row(status='Table TELCO_UNLABELED successfully created.')]

In [40]:
session.sql(f"use database {database}").collect()
session.sql(f"use schema {schema}").collect()

session.sql(f""" COPY INTO {database}.{schema}.TELCO_LABELED 
FROM '@{stage}/telco_labeled_subset.parquet' 
FILE_FORMAT = (TYPE = 'PARQUET')
MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE""").collect()

[Row(file='data_loading/telco_labeled_subset.parquet', status='LOADED', rows_parsed=90000, rows_loaded=90000, error_limit=1, errors_seen=0, first_error=None, first_error_line=None, first_error_character=None, first_error_column_name=None)]

In [41]:
session.sql(f"use database {database}").collect()
session.sql(f"use schema {schema}").collect()

session.sql(f""" COPY INTO {database}.{schema}.TELCO_UNLABELED 
FROM '@{stage}/telco_unlabeled_subset.parquet'
FILE_FORMAT = (TYPE = 'PARQUET')
MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE""").collect()

[Row(file='data_loading/telco_unlabeled_subset.parquet', status='LOADED', rows_parsed=9999, rows_loaded=9999, error_limit=1, errors_seen=0, first_error=None, first_error_line=None, first_error_character=None, first_error_column_name=None)]